In [ ]:
!pip install wget

In [ ]:
!pip install gluonnlp

In [ ]:
!pip install mxnet --user

In [ ]:
!pip install numpy  -U --user

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install nltk

In [ ]:
!pip install transformers

In [4]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

아레 실행 커멘드는 gpt_ckpt 폴더가 있지 않은 경우에만 실행해주세요.

In [5]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

100% [......................................................................] 460908853 / 460908853

In [6]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [7]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [8]:
BATCH_SIZE = 16
NUM_EPOCHS = 3
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [9]:
print(len(vocab))

50000


In [40]:
top_k = 1
filter_value=-99999
logits = tf.constant([1,2,3,4,5,6,7,8,9,10])
_logits = logits.numpy()
print(_logits)
temp = tf.math.top_k(logits, top_k)[0]
print(temp)
temp = temp[..., -1, None]
print(temp)
indices_to_remove = logits < temp
print(indices_to_remove)
_logits[indices_to_remove.numpy()] = filter_value
print(_logits)

[ 1  2  3  4  5  6  7  8  9 10]
tf.Tensor([10], shape=(1,), dtype=int32)
tf.Tensor([10], shape=(1,), dtype=int32)
tf.Tensor([ True  True  True  True  True  True  True  True  True False], shape=(10,), dtype=bool)
[-99999 -99999 -99999 -99999 -99999 -99999 -99999 -99999 -99999     10]


In [11]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:  # 확률이 높은 순으로 top_k의 단어는 선택에서 배제 된다.
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])


def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [29]:
generate_sent('이때', gpt_model, greedy=False)

"이때 그의 말대로에, 내 날개가  동방세력으로 분열되어  결국 그런지  모를  갖던 것을 '창조'라 하였다."

In [51]:
generate_sent('이때', gpt_model, top_k=3, top_p=0.0)

'이때쯤이면 이때가 오겠지?'

In [56]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.6)

'이때 애서 난 여기서 난 걸  보고 그애서                                                                                        '

In [60]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]
print(len(sents))

284


In [61]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token], padding = 'post')
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token], padding = 'post')

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [63]:
print(input_data[0])
print(output_data[0])

[    0 47437 47522 47675 47442 47437 47633 48120 47445 47441 47437 47455
 47467 48139 47445 47437 47676 47459 48090 47438 47453     3     3     3
     3     3     3     3     3     3]
[47437 47522 47675 47442 47437 47633 48120 47445 47441 47437 47455 47467
 48139 47445 47437 47676 47459 48090 47438 47453     1     3     3     3
     3     3     3     3     3     3]


In [64]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [65]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [66]:
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
16/16 [==============================] - 14s 238ms/step - loss: 4.3503 - accuracy_function: 0.1732 - val_loss: 2.6827 - val_accuracy_function: 0.2426
Epoch 2/3
16/16 [==============================] - 3s 196ms/step - loss: 2.8880 - accuracy_functio

In [ ]:
DATA_OUT_PATH = './data_out/KOR'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

In [71]:
generate_sent('이때', gpt_model, greedy=False)

'이때   가지는             ,그 애의        실응한    듯이                           ,눈결에    선지 는    았다.'

In [75]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때    가 메미의        가끔은                                                                                '